<a href="https://colab.research.google.com/github/jinseriouspark/polaris-llm/blob/main/ollama_gpu_running.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- colab으로 사용할 수 있는 gpu를 local 또는 다른 노트북에서 사용하기 위해 켜두는 코드입니다.
- [ngrok](https://ngrok.com/)에 계정을 생성하고, 무료 버전을 이용할 수 있는 토큰을 받아 cred.txt로 저장하고 실행하였습니다.
-

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh
!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
!sudo apt-get update && sudo apt-get install -y cuda-drivers
!pip install pyngrok


>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [872 kB]
Hit:8 https:

In [ ]:
from pyngrok import ngrok
with open('/content/drive/MyDrive/2024/keyword_extractor/cred.txt', 'r') as f:
  token = [r.strip() for r in f.readlines()]

In [ ]:
ngrok.set_auth_token(token[2])

In [ ]:
import os
import asyncio

# set LD_LIBRARY_PATH
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

In [ ]:
async def run_process(cmd):
  print('>>> starting', *cmd)
  p = await asyncio.subprocess.create_subprocess_exec(
      *cmd,
      stdout=asyncio.subprocess.PIPE,
      stderr=asyncio.subprocess.PIPE,
  )

  async def pipe(lines):
    async for line in lines:
      print(line.strip().decode('utf-8'))

  await asyncio.gather(
      pipe(p.stdout),
      pipe(p.stderr),
  )
from IPython.display import clear_output
clear_output()


await asyncio.gather(
run_process(['ollama', 'serve']),
run_process(['ngrok', 'http', '--log', 'stderr', '11434', '--host-header="localhost:11434"']),
run_process(['ollama', 'pull', 'gemma:7b-instruct']), # 사용할 모델 pull하는 부분
run_process(['ollama', 'pull', 'llama3:8b']),
run_process(['ollama', 'pull', 'wizardlm2:7b'])
)
